In [ ]:
from pyspark.sql import SparkSession

# ======================================
# 1. KHỞI TẠO SPARK SESSION
# ======================================

In [ ]:

spark = SparkSession.builder \
    .appName("VietnamWorks_JSON_Loader") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

print("Spark session started!")


# ======================================
# 2. ĐƯỜNG DẪN HDFS – sửa lại cho đúng của bạn
# ======================================

In [ ]:
# Ví dụ: hdfs://namenode:9000/user/hadoop/vnw/raw/vietnamworks_jobs.jsonl
hdfs_path = "hdfs://host.docker.internal:9000/user/hadoop/vietnamworks/vnw_raw.jsonl"

print(f"Reading from HDFS: {hdfs_path}")

# ======================================
# 3. ĐỌC DỮ LIỆU JSON/JSONL
# ======================================
# VietnamWorks JSON không phải JSONL chuẩn → multiline=True rất quan trọng

In [ ]:
df_raw = spark.read \
    .option("multiline", True) \
    .json(hdfs_path)

print("Schema:")
df_raw.printSchema()

print("\nSample records:")
df_raw.show(5, truncate=False)

In [ ]:
from pyspark.sql.functions import col

df_flat = df_raw.select(
    # Thông tin cơ bản
    col("url"),
    col("tieu_de").alias("job_title"),
    col("luong").alias("salary"),
    col("cong_ty").alias("company_name"),
    col("dia_diem").alias("location"),
    col("ngay_dang").alias("post_date"),
    col("ngay_dang_chi_tiet").alias("post_date_detail"),

    # Mô tả – yêu cầu
    col("mo_ta").alias("job_description"),
    col("yeu_cau").alias("requirements"),
    
    # Danh sách kỹ năng
    col("ky_nang").alias("skills_summary"),
    col("ky_nang_chi_tiet").alias("skills_detail"),

    # Phúc lợi (mảng)
    col("phuc_loi"),

    # Địa điểm chi tiết (mảng)
    col("dia_diem_chi_tiet"),

    # Các trường khác
    col("trang_thai").alias("status"),
    col("cap_bac").alias("level"),
    col("nganh_nghe").alias("career"),
    col("linh_vuc").alias("industry"),
    col("ngon_ngu").alias("language_requirement"),
    col("kinh_nghiem").alias("experience_years"),
    col("quoc_tich").alias("nationality_requirement"),
)

df_flat.printSchema()
df_flat.show(5, truncate=False)
